<a href="https://colab.research.google.com/github/Dirkster99/PyNotes/blob/master/PySpark_SparkNLP/02_articles_rowid_sentence_splitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install spark
# Guide from: https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Mount Google Drive to Load CSV File

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Colab\ Notebooks/Sentencing

'01 articles_rowid_converter.ipynb'	      articles.readme.txt.txt
'02 articles_rowid_sentence_splitter.ipynb'   articles_rowid.csv
 articles.csv


# Load Spark DataFrame and Split Sentences using Spark

1. Load Data from CSV File
1. Split Document Column into Array of Strings
1. Explode Array of Strings into Rows
1. Assign monotonically increasing id to support required order by in row_number() function
1. Apply row_number() function to enumerate an id per sentence in original order of documents and sentences in each document
 

In [ ]:
# Change this base directory if the CSV is located somewhere else
base_dir = '/content/drive/MyDrive/Colab Notebooks/Sentencing/'

In [ ]:
# Read the CSV file using Spark
df = spark.read.option("delimiter", "|").csv(base_dir + "articles_rowid.csv", header=True, inferSchema=True) \
.withColumnRenamed('row_id', 'rowid')

In [ ]:
df.printSchema()
df.head()

root
 |-- rowid: integer (nullable = true)
 |-- text: string (nullable = true)



Row(rowid=0, text='Die ARD-Tochter Degeto hat sich verpflichtet, ab August einer Quotenregelung zu folgen, die für die Gleichstellung von Regisseurinnen sorgen soll. In mindestens 20 Prozent der Filme, die die ARD-Tochter Degeto produziert oder mitfinanziert, sollen ab Mitte August Frauen Regie führen. Degeto-Chefin Christine Strobl folgt mit dieser Selbstverpflichtung der Forderung von Pro Quote Regie. Die Vereinigung von Regisseurinnen hatte im vergangenen Jahr eine Quotenregelung gefordert, um den weiblichen Filmschaffenden mehr Gehör und ökonomische Gleichstellung zu verschaffen. Pro Quote Regie kritisiert, dass, während rund 50 Prozent der Regie-Studierenden weiblich seien, der Anteil der Regisseurinnen bei Fernsehfilmen nur bei 13 bis 15 Prozent liege. In Österreich sieht die Situation ähnlich aus, auch hier wird von unterschiedlichen Seiten Handlungsbedarf angemahnt. Aber wie soll dieser aussehen? Ist die Einführung der Quotenregelung auch für die österreichische Film- und Ferns

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from pyspark.sql import types as T
from pyspark.sql import functions as F

udfSentencer = F.udf(lambda z: Sentencer(z), T.ArrayType(T.StringType()))
spark.udf.register("udfSentencer", colsInt)

# Convert text into an arry of strings where each item contains one sentence
def Sentencer(text):
  return sent_tokenize(text, language='german')

# Test Sentencer function with below code
#
#data = "All work and no play makes jack dull boy. What a joke if you think about it."
#print(Sentencer(data))

In [ ]:
tmpdf = df.withColumn( 'sentences', udfSentencer('text')).select('rowid', 'sentences')
tmpdf.printSchema()
tmpdf.head()

root
 |-- rowid: integer (nullable = true)
 |-- sentences: array (nullable = true)
 |    |-- element: string (containsNull = true)



Row(rowid=0, sentences=['Die ARD-Tochter Degeto hat sich verpflichtet, ab August einer Quotenregelung zu folgen, die für die Gleichstellung von Regisseurinnen sorgen soll.', 'In mindestens 20 Prozent der Filme, die die ARD-Tochter Degeto produziert oder mitfinanziert, sollen ab Mitte August Frauen Regie führen.', 'Degeto-Chefin Christine Strobl folgt mit dieser Selbstverpflichtung der Forderung von Pro Quote Regie.', 'Die Vereinigung von Regisseurinnen hatte im vergangenen Jahr eine Quotenregelung gefordert, um den weiblichen Filmschaffenden mehr Gehör und ökonomische Gleichstellung zu verschaffen.', 'Pro Quote Regie kritisiert, dass, während rund 50 Prozent der Regie-Studierenden weiblich seien, der Anteil der Regisseurinnen bei Fernsehfilmen nur bei 13 bis 15 Prozent liege.', 'In Österreich sieht die Situation ähnlich aus, auch hier wird von unterschiedlichen Seiten Handlungsbedarf angemahnt.', 'Aber wie soll dieser aussehen?', 'Ist die Einführung der Quotenregelung auch für die öste

In [ ]:
# Convert array of strings column into rows
# where each row contains one item from the array
df2 = tmpdf.select(tmpdf.rowid,F.explode(tmpdf.sentences))
df2 = df2.withColumn("rowid1", F.monotonically_increasing_id()) \
          .withColumnRenamed('col', 'sentence')                 \
          .select('rowid1', 'rowid', 'sentence')

df2.printSchema()
df2.show()

root
 |-- rowid1: long (nullable = false)
 |-- rowid: integer (nullable = true)
 |-- sentence: string (nullable = true)

+------+-----+--------------------+
|rowid1|rowid|            sentence|
+------+-----+--------------------+
|     0|    0|Die ARD-Tochter D...|
|     1|    0|In mindestens 20 ...|
|     2|    0|Degeto-Chefin Chr...|
|     3|    0|Die Vereinigung v...|
|     4|    0|Pro Quote Regie k...|
|     5|    0|In Österreich sie...|
|     6|    0|Aber wie soll die...|
|     7|    0|Ist die Einführun...|
|     8|    0|Diskutieren Sie i...|
|     9|    1|App sei nicht so ...|
|    10|    1|Wegen zu geringer...|
|    11|    1|Dafür soll der Li...|
|    12|    2|'Zum Welttag der ...|
|    13|    2|Denn erst Ausnahm...|
|    14|    2|Halbwissen ist of...|
|    15|    2|     Werther-Effekt?|
|    16|    2|Ja ja, schon mal ...|
|    17|    2|So wie damals, al...|
|    18|    2|Also: Über Suizid...|
|    19|    2|             Stimmt.|
+------+-----+--------------------+
only showing to

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number

df1=df2.withColumn("sent_num",row_number().over(Window.partitionBy("rowid").orderBy("rowid1"))) \
       .select('rowid', 'sent_num', 'sentence') \
       .orderBy('rowid', 'sent_num')

print("Printing the dataframe df1")
df1.show(n=50, truncate=False)

Printing the dataframe df1
+-----+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|rowid|sent_num|sentence                                                                                                                                                                                                                                                                                                                            |
+-----+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# NLTK Basics
https://pythonspot.com/tokenizing-words-and-sentences-with-nltk/

## Training for Abbreviations
https://nlpforhackers.io/splitting-text-into-sentences/

A sentence or data can be split into words using the method word_tokenize():

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

data = "All work and no play makes jack a dull boy, all work and no play"
print(word_tokenize(data))

['All', 'work', 'and', 'no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', ',', 'all', 'work', 'and', 'no', 'play']


# Tokenizing sentences

The same principle can be applied to sentences. Simply change the to **sent_tokenize()**
We have added two sentences to the variable data:

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

data = "All work and no play makes jack dull boy. What a joke if you think about it."
print(sent_tokenize(data))

['All work and no play makes jack dull boy.', 'What a joke if you think about it.']
